In [1]:
import sys,os
import torch
import pytorch_lightning as pl

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from models.dVAE.models.model import DiscreteVAE
from models.dVAE.training.lightning import DVAETrainModule

In [2]:
tensor = torch.zeros(1, 1, 64, 64)

dvae = DiscreteVAE(
    num_tokens=2048,
    codebook_dim=256,
    num_layers=1,
    num_resnet_blocks=2,
    hidden_dim=128,
    channels=1,
    smooth_l1_loss=True,
    temperature=0.2
)

model = DVAETrainModule.load_from_checkpoint('/workspace/Haea_dev/models/dVAE/vqvae_logs/2m_temperature/models/2m_temperature-epoch=31-val_loss=0.00.ckpt')




AttributeError: 'NoneType' object has no attribute 'temperature_scheduler'

In [36]:
checkpoint = torch.load('/workspace/Haea_dev/models/dVAE/vqvae_logs/geopotential/models/geopotential-epoch=00-val_loss=0.00.ckpt')
print(checkpoint.keys())
dvae.load_state_dict(checkpoint['dvae_state'])

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters', 'dvae_state'])


<All keys matched successfully>